## Scenario 1 - Base situation

Suppose there are "n" parallel coin toss games going on. Since, each game is similar in payoff structure (2:1) with 50% probability of win and has infinite future games ahead of it, we can assume that the proportion invested "f" remains constant over the games and is invested equally across the games without the loss of generality. Hence, each game gets f/n.

We can combine the outcomes of these "n" games each with two outcomes, into one single game with following (n+1) possible payoffs

$$ PY_i = (1+2*i*\cfrac{f}{n}-(n-i)*\cfrac{f}{n}) $$

with "i" ranging from 0 to n. Where "i" represents number of wins in "n" parallel games, f is the invested money combined in all games and each payoff having the following corresponding probability


$$ P_i = \cfrac{{n}\choose{i}}{2^n} $$

Suppose after sufficiently large N trials, each of the above payoffs have occurred k1, k2, k3, ..., kn times. Then geometric return per period is

$$ \prod_{i=0}^n PY_i^{\frac{ki}{N}} $$

As N tends to infinity, ki/N tends P<sub>i. Hence Geometric Return (GR) becomes
    

$$ \prod_{i=0}^n (1+2*i*\cfrac{f}{n}-(n-i)*\cfrac{f}{n})^{\cfrac{{n}\choose{i}}{2^n}} $$

Python code is used to maximize the above payoff by varying "f" from 0 to 1 for each "n" increasing from 1 to 100. Maximum geometric return is plotted on Y-axis with the number of parallel games (n) on X-axis and is shown in the figure below.

![fig_1](fig_1.png)

We can see that return per period is asymptotically leading to **1.5** and the geometric returns per period = **50%**.

#### Intuitive understanding of the result:

As the number of parallel games tend to infinity, number of wins and losses in each round will be equal to the expected values based on the "Law of Large Numbers". Since, it is a fair coin, number of wins and losses will be equal in each round.

Hence, the return will be equal to expected payoff = 0.5\*3+0.5\*0 = 1.5

As the risk of wealth getting completely wiped off in a single round becomes zero because there will be 50% of games winning each round, it is in our interest to invest the entire amount **(f = 1)** into the games as n --> infinity. There are **no drawdowns** as we get 50% fixed return every period. This can be observed in the following graphs.

![fig_2](fig_2.png)

![fig_3](fig_3.png)

In other words, this also means that payoff in infinite parallel game in this situation is similar to a **"Bond with 50% per period coupon"**. This is also corroborated by ~0 standard deviations obtained among even the 1-period payoffs from different random trials as seen in figure 3.

```python
# Importing Numpy and pandas for data storing and manipulation
import numpy as np
import pandas as pd

# to calculate the number of combination 
from scipy.special import comb

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
```

```python
# Base scenario: Using geometric return computations

# f: Combined amount invested in all games
f = np.arange(0, 1.01, 0.01)

# Creating a dataframe
# GRmaxac: Keeps track of max geometric returns
# fmaxac: keeps track of allocation which maximized geometric returns
df = pd.DataFrame(index=np.arange(1, 101), columns=['GRmaxac', 'fmaxac'])

# n: Number of parallel coin toss games
for n in range(1, 101):
    
    # initializing geometric returns by 1
    GR = 1
    
    # i: Number of wins in “n” parallel games
    for i in range(0,n+1):
        GR = GR * (1 + 2 * i * f / n - (n-i) * f / n)**(comb(n, i) / 2 ** n)
        
    # Find the max geometric returns    
    df.loc[n]['GRmaxac'] = max(GR)
    
    # Find the amount invested to get the max geometric returns
    df.loc[n]['fmaxac'] = f[np.argmax(GR)]
    
# Plot geometric returns and number of games in parallel w.r.t games
plt.scatter(df.index, df.GRmaxac)
plt.xlabel("Number of parallel coin toss games")
plt.ylabel("Maximum geometric returns")
plt.show()

# Plot amount invested and number of games in parallel w.r.t games
plt.scatter(df.index, df.fmaxac)
plt.xlabel("Number of parallel coin toss games")
plt.ylabel("Amount invested in all games")
plt.show()
```

```python
# Base scenario: Using 500 trials to estimate standard deviation

# f: Combined amount invested in all games
f = np.arange(0, 1.01, 0.01)

# Array to hold payoffs
payoffs = np.arange(1, 501)

# Creating a dataframe
# GRmax: Keeps track of returns
# varmax: keeps track of variances
df = pd.DataFrame(index=np.arange(1, 101), columns=['GRmax', 'varmax'])

# n: Number of parallel coin toss games
for n in range(1, 101):
    
    # Repeating 500 trials
    for tr in range(1, 501):
        
        result = 1
        for runs in range(1, 2):
            
            # tW: total wins
            tW = np.random.binomial(n, 0.5, 1)                        
            
            # tL: total losses
            tL = n - tW
            
            result = result * (1 + 2 * tW * f / n - tL * f / n)        
        
        payoffs[tr-1] = max(result ** (1 / runs))
    df.loc[n,'GRmax'] = np.mean(payoffs)
    df.loc[n,'varmax'] = np.var(payoffs)

# Plot variance w.r.t Number of parallel coin toss games played
plt.scatter(df.index, df.varmax)
plt.xlabel("Number of parallel coin toss games")
plt.ylabel("Maximum variance")
plt.show()
```

## Scenario 2 - Incorporating Risk of Ruin

As seen in the above scenario that the payoff from infinite parallel 2:1 game is a fixed 50% with ~0 standard deviation. There is no risk of ruin as there are no drawdowns from the wealth.

Assuming a Ruin of 60%, n = 100 parallel games, Probability of win = 0.5, Risk of Ruin is computed using 500 Monte Carlo simulations trails with 100 period run in each trail.

```python
# f: Amount invested in games
f = np.arange(0, 1.01, 0.01)

RR = np.arange(1, 101)
Ruin = 0.6

n = 100

count = 0.0
# Repeating 500 trails
for tr in range(1, 501):
    result = 1.0
    # 100 rounds
    for runs in range(1, 101):
        # tW: Represents total wins
        tW = np.random.binomial(n, 0.5, 1)                        
        # tL: Represents total losses
        tL = n - tW
        result = result * (1 + 2*tW*1/n - tL*1/n)          
        if result < Ruin:
            count = count + 1.0
            runs = 101

print(count/tr)
```

**Output**: Number of instances when there was a drawdown to the level of Ruin = 0

Hence, optimal allocations still remain the same as in the case of Base scenario where we invest the entire amount in the games and equally divide it between each of the games achieving 50% optimal geometric returns.

## Scenario 3 - Baised coin

Probability of win for each coin is varied from 0 to 1 and the variations in geometric return per period is observed with increasing number of parallel games as shown in figure 2.

Assuming probability of win is "p", following the same process as in the bases scenario, return per period becomes (Only the probability of each payoff changes)

$$ \prod_{i=0}^{n} (1+2*i*\frac{f}{n}-(n-i)*\frac{f}{n})^{{{i} \choose {n}} * p^i * (1-p)^{n-i}} $$

Where "i" representing number of wins in "n" parallel games, f is the invested money combined in all games.

![fig_4](fig_4.png)

#### Intuitive understanding of the result:

As we have established in scenario 1, optimal geometric returns are the expected value of return per round. This is re-affirmed in figure 2. Let probability of win is p. Assuming that we are investing, the optimal geometric return per period is 3\*p+0\*(1-p) = 3p.

For p < 1/3, this return is less than 1. Hence, it is better not to invest at all. As observed in the graph, optimal return per period remained at 1 for p = 0, 0.1, 0.2, 0.3.

For p > 1/3, returns per period = 3p is greater than 1. As observed in graph, in each of the lines, optimal returns are asymptotically leading to the value of  3\*winning probability.

```python
# Biased Coin

# f: Combined amount invested in all games
f = np.arange(0, 1.01, 0.01)

# Creating a dataframe
# GRmaxac: Keeps track of max geometric returns
# fmaxac: keeps track of allocation which maximized geometric returns
df = pd.DataFrame(index=np.arange(1, 101), columns=['GRmax', 'fmax'])

# Increase the plot size for better readibility
plt.figure(figsize=(10,7))

# Varying probability of winning from 0 to 1
# pW: Probability of win
for pW in np.arange(0, 1.1, 0.1):
    
    # pL: Probability of loss
    pL = 1-pW
    
    # n: Number of parallel coin toss games
    for n in np.arange(1, 101):
        GR = 1
        for i in np.arange(1, n+1):
            GR = GR * (1 + 2 * i * f / n - (n - i) * f / n)**(comb(n, i)*pW**i*pL**(n-i))
        
        df.loc[n,'GRmax'] = max(GR)
        df.loc[n,'fmax'] = f[np.argmax(GR)]
    plt.plot(df.index, df.GRmax, label=round(pW,1))
    
plt.legend(loc='best')
plt.xlabel("Number of parallel coin toss games")
plt.ylabel("Maximum geometric returns")
plt.show()
```

## Scenario 4 - Interest earned on the unused money

From the base scenario, interest rate (rf) earned on the uninvested money is varied from 0% to 50% and the variation in geometric return per period is observed with increasing number of parallel games as shown in figure 3.

Probability of win is fixed as 0.5. For an interest rate of rf, geometric return per period (GR) becomes

$$ \prod_{i=0}^{n}(1+2*i* \frac{f}{n} - (n-i) * \frac{f}{n} + rf*(1-f))^{\frac{n \choose i}{2^n}} $$

Where "i" representing number of wins in "n" parallel games, f is the invested money combined in all games.

#### Intuitive understanding of the result:

Since, in the optimal situation maximizing Geometric returns per period in the base Scenario, the entire amount (f=1) is equally invested in all the infinite parallel games, there should be no significant effect of the interest rate as there is no uninvested money and invested money in games is generating 50% returns per period. Hence, for interest rates lower than 50%, the optimal allocations and maximum returns do not change for higher “n”s as observed in Figure 3 when rf < 50%. Only for lower n’s, there is an effect of increasing interest rate on optimal returns.

![fig_5](fig_5.png)

When rf >= 50% (very very unreal), none of the wealth should be invested in the games (f=0) as the expected return from the games is only tending to 50%. Then the entire wealth is held as cash which will be yielding returns of rf (>= 50%).


```python
# f: Amount invested in games
f = np.arange(0, 1.01, 0.01)
df = pd.DataFrame(index=np.arange(1, 101), columns=['GRmax', 'fmax'])

plt.figure(figsize=(10,7))
# Various interest rates from 10% to 50%
for rf in np.arange(0.1, 0.6, 0.1):
    for n in range(1, 101):
        GR = 1
        for i in np.arange(0, n+1):
            GR = GR * (1 + 2 * i * f / n - (n - i) * f / n + rf * (1 - f)) ** (comb(n, i) / 2 ** n)
            
        df.loc[n,'GRmax'] = max(GR)
        df.loc[n,'fmax'] = f[np.argmax(GR)]
    plt.plot(df.index, df.GRmax, label=round(rf,1))
    
plt.legend(loc='best')
plt.xlabel("Number of parallel coin toss games")
plt.ylabel("Maximum geometric returns")
plt.show()
```

## Scenario 5 - Different Probability of runs/reversals

Probability of runs (PR) is defined as Probability of win after win or Probability of loss after loss. This PR is varied from 0 till 1 (low PR implying high probability of reversal and high PR implying high probability of continuation/run) and the maximum Geometric return per period is observed for n = 500 parallel games in each round.

For the first game, the probabilities are assumed to be 0.5 (final output is not dependent on this). After the first round, there will be two types of games – ones which have won in first round and the ones which have lost and both of them have different probabilities of wins for this round. As seen from previous scenarios, because of infinite games in parallel, it is optimal to invest entirely in the games. The amount of investment in the games that have won in last games is assumed to be “f” and the amount in lost games is “1-f”. These amounts will be equally divided among games within each of these sets.  Since, there will be infinite set of games in each set (Won/Lost), returns per set will be uniform across periods. Hence, the allocation “f” and “1-f” is assumed to remain the same post period 2. Using 500 Monte Carlo simulation trials and by varying f, average maximum return per period is computed. The findings are as below.

|Probability of Run(PR)|0/1|0.1/0.9|0.2/0.8|0.3/0.7|0.4/0.6|0.5|
|----------------------|---|-------|-------|-------|-------|---|
|Maximum Return per period|3|2.7|2.4|2.1|1.8|1.5|

And the optimal allocations obtained are when we invest entirely in the set of games which we have a higher probability of Win for the next game (depends on PR).

#### Intuitive understanding of the result:

Since, we know the probability of Win and loss are different based on last round, we can categorize all games into two sets – One with higher Win % and the other with lower Win % based on Probability of Run. Since there is a non-zero chance of a Win/Loss occurring, there will be infinite games in each of the categories too. Hence, the expected returns from each
category are going to 3\*Prob of Win of that category.

To maximize returns, I’ll only put my entire amount in the category which has a higher Win % (as observed in the R output) and divide the wealth into equal proportions based on the number of games present in that category. Hence, Expected Returns per period = max (3 \* PR, 3 \* (1-PR)) which is as observed in the above table.

Variance of the returns is also estimated to re-corroborate the similarity between these infinite parallel 2:1 games and payoff from a fixed rate bond. All the variances for different probability of runs came close to 0.

```python
# Number of parallel games
n = 500

# f: Amount invested in games
f = np.arange(0, 1.01, 0.01)

df = pd.DataFrame(index=np.arange(1, 501),columns=['payoffs'])

print(['Probability of Run', 'Maximum Returns', 'Varinace of Returns'])

for pp in np.arange(0, 1.1, 0.1):       
    
    # Repeating 500 times
    for tr in np.arange(1, 501):
        result = 1
        # tW: Represents total wins in the last round
        tW = np.random.binomial(n, 0.9, 1)
        
        # tL: Represents total losses in the last round
        tL = n - tW
        
        result = result * (1+2*tW*1/n-tL*1/n)
        
        # 500 rounds
        for runs in np.arange(1, 501):
            # WW: Number of wins after a win in the last round
            WW = np.random.binomial(tW, pp, 1)
            WL = tW - WW
            
            # LW: Number of wins after a loss in the last round
            LW = np.random.binomial(tL, 1-pp, 1)
            LL = tL-LW
            
            result = result *((3.0*WW)*f/tW+(3.0*LW)*(1.0-f)/tL)            
            tW = WW+LW
            tL = WL+LL
        df.loc[tr,'payoffs'] = np.max(result**(1.0/501.0))
        
    col = [round(pp,1), round(np.mean(df.payoffs),2), round(np.var(df.payoffs),2)]    
    print(col)

```